In [1]:
from common import *
from features.selector import Function
from features.FFF import FFF
from features.template import fill_template
from features.FTP_solo import soloTemplate, soloEdit
from features.FTP_pair import copyTemplate, copyEdit, diffTemplate, diffEdit, corrTemplate, corrEdit

In [2]:
from given import make_CC_G_Q_from_Dl_F_P_W, make_FG_PQ_from_CC_G_Q, setup_P

In [3]:
_TEST_ = False

In [4]:
if __name__=='__main__':
    _TEST_ = True
    interactive(rows=2000, cols=2000)

# defining the package exports

In [7]:
class Features():
    def __init__(self, **kw):
        for k, v in kw.items():
            setattr(self, k, v)
    
    def gen_solo(self, i):
        self.X, self.Y = self.F, self.P
        
        feats = self.F.columns
        fffTemplate, fffEdit = soloTemplate, soloEdit

        _r = random.Random(i)
        fffEdit._random_data = (_r.random(), _r.random(), _r.random())
        while True:
            q = '\n'.join(fill_template(fffTemplate.splitlines(), lambda x: x-_r.random(), fffEdit))
            f = list(filter(FFF.query(q), feats))
            if 11 <= len(f) < 159:
                return f, q
        
    def gen_pair(self, i):
        if hasattr(self, '_cache_i') and self._cache_i==i:
            return list(self.X.columns), self._cache_q
        
        feats = self.F.columns

        _r = random.Random(i)
        copyEdit._random_data = (_r.random(), _r.random(), _r.random())
        diffEdit._random_data = (_r.random(), _r.random(), _r.random())
        
        q_, filt_ = [], []
        for fffTemplate, fffEdit in [(copyTemplate,copyEdit),(diffTemplate,diffEdit),(copyTemplate,copyEdit)]:
            while True:
                q = '\n'.join(fill_template(fffTemplate.splitlines(), lambda x: x-_r.random(), fffEdit))
                filt = FFF.query(q)
                if 6 <= len(list(filter(filt, feats))) < 159:
                    q_.append(q)
                    filt_.append(filt)
                    break
                    
        qCorr = '\n'.join(fill_template(corrTemplate.splitlines(), lambda x: x-_r.random(), corrEdit))
        filtCorr = FFF.query(qCorr)
        
        # logic to select what features to keep
        while True:
            hasCopy = _r.random() < .85
            hasDiff = _r.random() < .5
            diffUseCopy = _r.random() < .15
            hasSign = _r.random() < .4
            signUseCopy = _r.random() < .2
            if hasCopy or hasDiff or hasSign:
                break
        
        filtCopy = Function(lambda x: False) if not hasCopy else filt_[0]
        qCopy = '0' if not hasCopy else q_[0]
        filtDiff = Function(lambda x: False) if not hasDiff else filt_[0] if diffUseCopy else filt_[1]
        qDiff = '0' if not hasDiff else q_[0] if diffUseCopy else q_[1]
        filtSign = Function(lambda x: False) if not hasSign else filt_[0] if signUseCopy else filt_[2]
        qSign = '0' if not hasSign else q_[0] if signUseCopy else q_[2]
        # end logic to select features to keep
            
        feats = list(filter(filtCopy|filtDiff|filtSign, feats))
        if 'assetCodeId' not in feats:
            feats = ['assetCodeId'] + feats
            
        CC, G, Q = make_CC_G_Q_from_Dl_F_P_W(
            Dl = self.D.av10,
            F = self.F[feats],
            P = self.P,
            W = self.W,
            lo = self.Pair.lo,
            hi = self.Pair.hi,
            repeat = self.Pair.repeat
        )
        FG, PQ = make_FG_PQ_from_CC_G_Q(filtCopy, filtDiff, filtSign, filtCorr, CC=CC, G=G, Q=Q)
        
        q = qCopy+'\n#<>#\n'+qDiff+'\n#<>#\n'+qSign+'\n#<>#\n'+qCorr
        self._cache_i, self.X, self.Y, self._cache_q = i, FG, PQ, q
        return list(FG.columns), q

# .................................. / Scratcher

In [ ]:
# import json
# #json.dump(list(F.columns), open('feats.json', 'w'))
# feats = json.load(open(top_dir/'work/feats.json', 'r'))

In [8]:
# (lambda x: print(len(x[0]), '\n\n', x[0], '\n\n', x[1]))(gen_features(10))

70 

 ['volume', 'open', 'acc', 'oo10', 'oo15', '(oo-aoo)10', '(oo-aoo)15', 'volatility60', 'oo(10-5)', 'oo(20-15)', 'oo(15-5)', 'oo(20-5)', '(oo-aoo)(10-5)', '(oo-aoo)(20-15)', '(oo-aoo)(15-5)', '(oo-aoo)(20-5)', '(oo-cc){0/1}', '(oo-cc){0/2}', '(oo-cc){1/2}', '(oo-cc){2}', '(it,af,it{1},aoo,acc){0/1}', '(it{1},af{1},it{2},aoo{1},acc{1}){0/2}', '(it,af,it{1},aoo,acc){0/3}', '(it,af,it{1},aoo,acc){1/2}', '(it{1},af{1},it{2},aoo{1},acc{1}){1/2}', '(it,af,it{1},aoo,acc){1/4}', '(it{1},af{1},it{2},aoo{1},acc{1}){2/4}', 'oo1Since125', 'oo1Drawdown1Since125', 'oo1Drawdown(20-10)Since125', 'oo1MinSince125{5}', 'oo1MinSince125{20}', 'oo10Since125', 'oo10Drawdown1Since125', 'oo10Drawdown(20-10)Since125', 'oo10MinSince125{5}', 'oo10MinSince125{20}', 'volatility10Drawdown(20-10)Since125', 'volatility10MaxSince125{10}', 'volatility10MaxSince125{20}', 'volatility10MinSince125{20}', 'volatility20Drawdown(20-10)Since125', 'volatility20MaxSince125{10}', 'volatility20MaxSince125{20}', 'volatility20Min